In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [8]:
train_data = pd.read_csv("train.csv")
train_data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [9]:
test_data = pd.read_csv("test.csv")
test_data.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


## Ramdom Forest

In [23]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch",]
X = pd.get_dummies(train_data[features])
X = X.dropna()
# train_Age = pd.DataFrame({"Age" : train_data["Age"]})
# X = pd.concat([X_pre, train_Age], axis = 1)
X_test = pd.get_dummies(test_data[features])
X_test = X_test.dropna()
# test_Age = pd.DataFrame({"Age" : test_data["Age"]})
# X_test = pd.concat([X_test_pre, test_Age], axis = 1)

from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 4, 5, 6, 7]
}

model = GridSearchCV(RandomForestClassifier(n_estimators=100, random_state=1), param_grid,  cv=5)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
# output.to_csv('submission.csv', index=False)
print('CV score: {}'.format(model.score(X, y))) 
print('Best parameters: {}'.format(model.best_params_))

CV score: 0.8148148148148148
Best parameters: {'max_depth': 4}


In [16]:
X.head(3)

,Pclass,SibSp,Parch,Age,Sex_female,Sex_male
0,3,1,0,22.0,False,True
1,1,1,0,38.0,True,False
2,3,0,0,26.0,True,False


## NN

In [31]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

### Batches

### Define networks

In [ ]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.cnn_layer = nn.Sequential(
            nn.Conv2d()
        )

In [35]:
yyy = \
    100
yyy + 2

102

## Gradient Boosting tree

In [38]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# データの分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)



### Prediction of "Age"

In [52]:
from sklearn.ensemble import RandomForestRegressor

# 推定に使用する項目を指定
X = train_data[['Age', 'Pclass','Sex','Parch','SibSp']]

# ラベル特徴量をワンホットエンコーディング(数量化)
X =pd.get_dummies(X)

# 学習データとテストデータに分離し、numpyに変換
known_age = X[X.Age.notnull()].values  
unknown_age = X[X.Age.isnull()].values

# 学習データをX(Ageの説明変数), y(Age Unknown)に分離
X_age = known_age[:, 1:]  
y_age = known_age[:, 0]

model_age = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1)
model_age.fit(X_age, y_age)

# 推定モデルを使って、テストデータのAgeを予測し、補完
predictedAges = model_age.predict(unknown_age[:, 1::])
X.loc[(X.Age.isnull()), 'Age'] = predictedAges



In [40]:
import lightgbm as lgb

# データの分割
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBMモデルの定義
model = lgb.LGBMClassifier()

# モデルの学習
model.fit(X, y)

# バリデーションデータでの予測
y_pred = model.predict(X)

# 精度の評価
# accuracy = accuracy_score(y_val, y_pred)
# print(accuracy)

[LightGBM] [Info] Number of positive: 268, number of negative: 444
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000972 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.376404 -> initscore=-0.504838
[LightGBM] [Info] Start training from score -0.504838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [42]:
prediction = model.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': prediction})
output.to_csv('submission.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'submission.csv'